In [1]:
import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium_stealth import stealth
import random
import time
import os
import unidecode

Scrap lien drive

In [2]:
def scrap_lien_drive(driver,list_lien_drives =[]):

    driver.get("https://www.casino.fr/prehome/courses-en-ligne/magasins")
    time.sleep(2)
    


    button = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, 'onetrust-reject-all-handler'))
        )
    button.click()
    time.sleep(2)
    

    for i in range(2,90,1):
        for j in range(1,50,1):
            try : 
                lien_drive = driver.find_element(By.XPATH, f'html/body/div[1]/div[1]/div[4]/div/div[{i}]/div/div[{j}]/div/div/div[2]/a[1]')
                try : 
                    for k in range(1,50,1):
                        lien_drive = driver.find_element(By.XPATH, f'html/body/div[1]/div[1]/div[4]/div/div[{i}]/div/div[{j}]/div/div[{k}]/div[2]/a[1]')
                        lien_drive = lien_drive.get_attribute("href")
                        print(lien_drive)
                        list_lien_drives.append(lien_drive)
                except :
                    pass
                lien_drive = lien_drive.get_attribute("href")
                #print(lien_drive)
                list_lien_drives.append(lien_drive)
            except:
                pass

    
    return(list_lien_drives)

Scrap attribut drive

In [3]:
def scrap_drive(driver,list_lien_drive,list_drive=[]):
    for lien in list_lien_drive:
        try :
            driver.get(lien)
            
            nom_drive = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'store-header__name')))
            
            adress_drive = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'store-body__location-detail')))

            lien_drive = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[2]/div/div/a')))
            
            lien_drive = lien_drive.get_attribute("href")

            lien_coord = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[3]/div[2]/a')))
            lien_coord = lien_coord.get_attribute("href")

            dict_drive ={
                "nom_drive": nom_drive.text,
                'adresse': adress_drive.text,
                "lien_drive": lien_drive,
                'lien_coord':lien_coord
            }
            #print(dict_drive)
            list_drive.append(dict_drive)
        except : 
            pass
            
            
     
    df_drive = pd.DataFrame(list_drive)
    df_drive.to_csv("list_drives_casino.csv")
    
    return(df_drive)
        

find last index of a specific character in string 

In [4]:
def find_last_index(string, char, indices = []):
    for i in range(len(string)):
        if string[i] == char:
            indices.append(i)

    last_indice = int(indices[-1])
    return(last_indice)

In [5]:
def find_first_index(string, char, indices = []):
    for i in range(len(string)):
        if string[i] == char:
            indices.append(i)

    last_indice = int(indices[-2])
    return(last_indice)

DRIVE NETTOYé

In [6]:
def process_drive(df,list_drives = []):
    for i,adresse in enumerate(df["adresse"]):

        lastVirgule = find_last_index(string = adresse, char=',')
        

        city = adresse[lastVirgule-5:].replace(",","")
        department = adresse[lastVirgule-5:lastVirgule-3]
        

        lienCoord= df.iloc[i]['lien_coord']
        firstEt = find_first_index(string = lienCoord, char='&')
        percent = find_last_index(string = lienCoord, char='%')
        lastEt = find_last_index(string = lienCoord, char='&')

        latitude = lienCoord[firstEt+7:percent]
        longitude = lienCoord[percent+3:lastEt]
        
        reservedCharacter = ["!","*","'","(",")",";",":","@","&","=","+","$",",","/","?","%","#","[","]"," "]
        nom_driveUrl = df.iloc[i]['nom_drive']
        for character in reservedCharacter:
            nom_driveUrl = nom_driveUrl.replace(character,"_")
        nom_driveUrl = unidecode.unidecode(nom_driveUrl)
        dict_drive ={
            "supermarket":"Casino",
            "department": department,
            "nom_drive": df.iloc[i]['nom_drive'],
            "nom_driveUrl" : nom_driveUrl,
            'city': city,
            "adresse": adresse.replace(",",""),
            "latitude": latitude,
            "longitude": longitude,
            "lien_drive" : df.iloc[i]['lien_drive'],
            "lien_coord" : df.iloc[i]['lien_coord']

        }
        list_drives.append(dict_drive)
        print(dict_drive)
    df_drives = pd.DataFrame(list_drives)
    dropSuper = ["SPAR", "Vival",'shop','Petit','hyperFrais',
                'PARIS Saint-Didier','FERRIERES-EN-BRIE','FONTAINEBLEAU','NEMOURS','VULAINES-SUR-SEINE','ANDRESY','Hardricourt','Maisons-Laffitte',
                'Bagneux','Chatenay Malabry','CHAVILLE','CLICHY Martre','SURESNES','Le Blanc Mesni','LES-LILAS','MAISONS-ALFORT','LA-VARENNE-SAINT-HILAIRE','Villejuif'
                 ]
    df_drives = df_drives[~df_drives['nom_drive'].str.contains('|'.join(dropSuper))]
    df_drives.to_csv("list_drives_casino.csv")


In [7]:
#set up driver
# create a new Service instance and specify path to Chromedriver executable
service = ChromeService(executable_path=ChromeDriverManager().install())


# Step 2: Change browser properties
# create a ChromeOptions object
options = webdriver.ChromeOptions()

#run in headless mode
#options.add_argument("--headless")

# disable the AutomationControlled feature of Blink rendering engine
options.add_argument('--disable-blink-features=AutomationControlled')
 
# disable pop-up blocking
options.add_argument('--disable-popup-blocking')
 
# start the browser window in maximized mode
options.add_argument('--start-maximized')
 
# disable extensions
options.add_argument('--disable-extensions')
 
# disable sandbox mode
options.add_argument('--no-sandbox')
 
# disable shared memory usage
options.add_argument('--disable-dev-shm-usage')


# Set navigator.webdriver to undefined
# create a driver instance
driver = webdriver.Chrome(service=service, options=options)

# Change the property value of the navigator for webdriver to undefined
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")



# Step 3: Rotate user agents 
user_agents = [
    # Add your list of user agents here
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
]

# select random user agent
user_agent = random.choice(user_agents)

# pass in selected user agent as an argument
options.add_argument(f'user-agent={user_agent}')

# set user agent using execute_cpd_cmd
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent})


# Step 4: Scrape using Stealth
#enable stealth mode
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )
    
list_lien_drive = scrap_lien_drive(driver)
df=scrap_drive(driver,list_lien_drive)
df = pd.read_csv('list_drives_casino.csv')
process_drive(df)
    

JSONDecodeError: Unterminated string starting at: line 1 column 1269756 (char 1269755)